In [20]:
import netrustpy3
import networkx as nx

In [21]:
#get sample graph from networkx
G = nx.karate_club_graph()
edge_list = list(G.edges())
print(f"number of nodes: {G.number_of_nodes()}")
print(f"number of edges: {G.number_of_edges()}")

number of nodes: 34
number of edges: 78


In [22]:
G_rust = netrustpy3.Graph.from_edge_list(edge_list)

print(f"Number of nodes: {G_rust.num_v()}")
print(f"Number of edges: {G_rust.num_e()}")

Number of nodes: 34
Number of edges: 78


In [24]:
%%timeit
bfs_tree = nx.bfs_tree(G, 0)

62.2 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [23]:
%%timeit
bfs_tree = netrustpy3.bfs(G_rust, 0)

82.7 µs ± 3.8 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [17]:
bfs_tree = netrustpy3.bfs(G_rust, 0)

In [18]:
print(f"distance from node 1 to node 3: {bfs_tree[3][1]}")

# get path from node 1 to node 3
path = []
node = 3
while node != None:
    path.append(node)
    node = bfs_tree[node][0]

print(f"path from node 1 to node 3: {path[::-1]}")

distance from node 1 to node 3: 1
path from node 1 to node 3: [0, 3]


[3, 2]